In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

In [3]:
os.chdir('drive/MyDrive/Colab Notebooks/')

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd
import cv2
import os
import pickle
from skimage.color import rgb2gray
from PIL import Image
from google.colab.patches import cv2_imshow
import sys

In [7]:
def detect_jersey_number(img):
  # cv2_imshow(img)
  bboxes = []
  scale_percent = 500

  imh = img.shape[0]
  imw = img.shape[1]

  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
    
  # resize image
  img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

  hImg = img.shape[0]
  wImg = img.shape[1]

  #crop
  img = img[hImg//10*2:hImg//10*5, wImg//100*10:wImg//100*90]

  #grayscale
  gray = ~ cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  #blur
  blur = cv2.GaussianBlur(gray, (7,7), 0)

  #threshold
  thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,13))

  #dilate
  dilate = cv2.dilate(thresh, kernel, iterations=1)

  #contours
  cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  cnts = sorted(cnts, key=lambda x:cv2.boundingRect(x)[0])
  
  for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if h>img.shape[0]//10*3 and w>img.shape[0]//10*3 and h<img.shape[0]//10*7 :
      bboxes.append(((x+imw//2)//5,(y+imh)//5,w//5,h//5))

  return bboxes

In [9]:
digit_recognition_model = load_model('number_detection_model.h5')

In [14]:
def identify_number(img):
  digits=[]
  nr=0

  cropped_img=img
  width = int(cropped_img.shape[1] * 5)
  height = int(cropped_img.shape[0] * 5)
  dim = (width, height)
    
  cropped_img = cv2.resize(cropped_img, (dim), interpolation = cv2.INTER_AREA)
  
  gray = ~ cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
  
  blur = cv2.GaussianBlur(gray, (7,7), 0)

  thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,13))

  dilate = cv2.dilate(thresh, kernel, iterations=1)

  cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]

  cnts = sorted(cnts, key=lambda x:cv2.boundingRect(x)[0])
  for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if h>cropped_img.shape[0]//10*5 and w>cropped_img.shape[0]//4:
      
      digit_img = cropped_img[y:y+h, x:x+w]
      
      gray = cv2.cvtColor(digit_img, cv2.COLOR_BGR2GRAY)
      blur = cv2.GaussianBlur(gray, (7,7), 0)
      thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
      
      digit_black_and_white_img = cv2.resize(thresh, (20,20), interpolation = cv2.INTER_AREA)
      digit_black_and_white_img = cv2.copyMakeBorder(digit_black_and_white_img, 4, 4, 4, 4, cv2.BORDER_CONSTANT,value=[255, 255, 255])

      digit_black_and_white_img = digit_black_and_white_img/255

      digit_black_and_white_img = np.expand_dims(digit_black_and_white_img, (0,3))

      pred = digit_recognition_model.predict(digit_black_and_white_img)
      
      if pred[0][pred.argmax()]>0.7:
        digits.append(pred.argmax())

  number = "".join([str(digit) for digit in digits])
  return number

In [18]:
def count_nonblack_pixels(img):
    return img.any(axis=-1).sum()

In [19]:
def detect_team(image):

    color_list=['red','yellow','blue']
    ratios = []

    boundaries = [
    ([17, 15, 100], [50, 56, 200]), #red
    ([25, 146, 190], [96, 174, 250]), #yellow
    ([43, 31, 4], [250, 88, 50]), #blue
    ]
    
    i = 0
    for (lower, upper) in boundaries:

        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")


        mask = cv2.inRange(image, lower, upper)
        output = cv2.bitwise_and(image, image, mask = mask)
        tot_pix = count_nonblack_pixels(image)
        color_pix = count_nonblack_pixels(output)
        ratio = color_pix/tot_pix
        ratios.append(ratio)
        i += 1

    if np.max(ratios) < 0.015:
      return 'not sure'   
    return color_list[np.argmax(ratios)]

In [ ]:
writer = None

h, w = None, None

min_prob = 0.5

colors={
    'blue':(255,0,0),
    'red':(0,0,255),
    'green':(0,255,0),
    'yellow':(0,255,255),
}

video = cv2.VideoCapture('data/input_video.mp4')

network = cv2.dnn.readNet('yolov3.weights','darknet/cfg/yolov3.cfg')
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

ln = network.getLayerNames()
ln = [ln[i - 1] for i in network.getUnconnectedOutLayers()]

while True:
  ret, frame = video.read()
  if not ret:
      break
  

  boxes = []
  box_colors = []
  bboxes={}
  confidences = []
  classIDs = []
  h, w = frame.shape[:2]
  
  blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
  network.setInput(blob)
  outputs = network.forward(ln)

  for output in outputs:
      for detection in output:
          scores = detection[5:]
          classID = np.argmax(scores)
          confidence = scores[classID]


          if confidence > 0.5:
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            box = [x, y, int(width), int(height)]

            player_image = frame[y:y+int(height), x:x+int(width)]
            if player_image.shape[0] == 0 or player_image.shape[1] == 0 or player_image.shape[2] == 0:
                    continue
            boxes.append(box)
            confidences.append(float(confidence))
            classIDs.append(classID)

            color = detect_team(player_image)
            box_colors.append(color) 
            try:
              number_bboxes = detect_jersey_number(player_image)
              for b in number_bboxes:
                X,Y,W,H = b[0], b[1], b[2], b[3]
                number = identify_number(frame[y+Y:y+Y+H, x+X:x+X+W])
                cv2.rectangle(frame, (x+X,y+Y), (x+X+W, y+Y+H), (36,255,12), 2)
                cv2.putText(frame, f'Number {number}',(x-10, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
              bboxes[len(boxes)-1] = number_bboxes
            except Exception:
              pass
            
  indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  
  if len(indices) > 0:
    for i in indices.flatten():
          box = boxes[i]

          if classIDs[i] == 0:
              color = (0, 0, 0)
              text = 'GK/REF'
              if box_colors[i] != 'not sure':
                color = colors[box_colors[i]]
              if box_colors[i] == 'red':
                text = 'Belgium'
              if box_colors[i] == 'blue':
                text = 'France'
              if box_colors[i] == 'yellow':
                text = 'GK'

              cv2.rectangle(frame, (round(box[0]),round(box[1])), (round(box[0]+box[2]),round(box[1]+box[3])), color, 2)
              cv2.putText(frame, text, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


  if writer is None:
        resultVideo = cv2.VideoWriter_fourcc(*'mp4v')

        
        writer = cv2.VideoWriter('data/output_video.mp4', resultVideo, 24.0,
                                (frame.shape[1], frame.shape[0]), True)

  
  writer.write(frame)


video.release()
writer.release()